In [63]:
import pandas as pd
import json

import utils as ut

In [64]:
df = pd.read_csv('../backups/run5/cdt_cam.csv')
df = ut.extract_cols(df)

In [65]:
df.head()

,TPR_pattern,FPR_pattern,FPRn_pattern,SHD_pattern,SHD_cpdag,FPR_skel,FNR_skel,TP_skel,FN_skel,FP_skel,...,cutoff,variablesel,selmethod,pruning,prunmethod,timeout,graph_p,graph_d,data_sem,data_n
0,0.500000,0.175000,0.700000,12,13,0.400000,0.200000,8,2,4,...,0.001,True,gamboost,True,gam,NaN,10,1,gumbel,200
1,0.650000,0.262500,1.050000,14,14,0.700000,0.000000,10,0,7,...,0.001,True,gamboost,True,gam,NaN,10,1,gumbel,1000
2,0.445946,0.730769,0.256757,30,29,0.108108,0.405405,22,15,4,...,0.001,True,gamboost,True,gam,NaN,10,4,gumbel,200
3,0.432432,0.846154,0.297297,32,32,0.108108,0.378378,23,14,4,...,0.001,True,gamboost,True,gam,NaN,10,4,gumbel,1000
4,0.608696,0.090395,0.695652,25,26,0.434783,0.130435,20,3,10,...,0.001,True,gamboost,True,gam,NaN,20,1,gumbel,200


In [34]:
with open('../config/algos_repo.json', 'r') as f:
    config = json.load(f)

In [35]:
obj = config['resources']['structure_learning_algorithms']
for alg in obj:
    print(obj[alg])

[{'id': 'pc-gaussCItest', 'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5], 'NAdelete': True, 'mmax': 'Inf', 'u2pd': 'relaxed', 'skelmethod': 'stable', 'conservative': False, 'majrule': False, 'solveconfl': False, 'numCores': 1, 'verbose': False, 'indepTest': 'gaussCItest', 'timeout': None}]
[{'id': 'fges-sem-bic', 'faithfulnessAssumed': True, 'score': 'sem-bic', 'datatype': 'continuous', 'samplePrior': 1, 'structurePrior': 1, 'penaltyDiscount': [0.1, 0.25, 0.5, 0.75, 1, 1.5], 'timeout': None}]
[{'id': 'fci-fisher-z', 'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5], 'test': 'fisher-z-test', 'datatype': 'continuous', 'timeout': None}]
[{'id': 'gcastle_ica_lingam', 'thresh': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5], 'random_state': None, 'max_iter': [100, 1000], 'timeout': None}]
[{'id': 'gcastle_anm', 'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5], 'timeout': None}]
[{'id': 'cam', 'score': 'nonlinear', 'cutoff': [0.001, 0.01, 0.1], 'variablesel': True, 'selmethod': 'gamboost', 'pruning': True, 'prunm

In [43]:
params = list(obj['cdt_cam'][0].keys())

In [66]:
df['hyper_id'] = df.groupby(params, dropna=False).ngroup()

In [107]:
#gr = df.groupby(['graph_p', 'graph_d', 'graph_type', 'data_sem', 'data_n', 'hyper_id'])['SHD_pattern'].agg(['mean', 'sem'])
#gr = df.groupby(['data_sem', 'hyper_id'])['SHD_pattern'].agg(['mean', 'sem'])
gr = df.groupby(['hyper_id'])['SHD_pattern'].agg(['mean', 'sem'])

In [108]:
gr = gr.reset_index()

In [112]:
gr

,hyper_id,mean,sem
0,0,58.241667,3.282940
1,1,58.760417,3.297791
2,2,60.343750,3.337265
3,3,62.604167,3.379092
4,4,66.131250,3.444996
5,5,74.354167,3.601909
6,6,84.635417,3.839674
7,7,98.129167,4.211672
8,8,106.868750,4.446213
9,9,116.550000,4.650436


In [115]:
#gr.apply(lambda x: x.loc[x['mean'].idxmin()])
hyper_id_sel = gr.loc[gr['mean'].idxmin(), 'hyper_id']

In [95]:
gr

,data_sem,hyper_id,mean,sem
0,gp,1,44.704167,3.619493
1,gumbel,0,71.716667,5.335022


In [116]:
#df2 = df.join(gr.set_index('data_sem'), on='data_sem', rsuffix='_sel')
df['hyper_id_sel'] = hyper_id_sel

In [117]:
df2 = df.loc[df['hyper_id'] == df['hyper_id_sel']]

In [118]:
df2.groupby(['graph_p', 'graph_d', 'graph_type', 'data_sem', 'data_n'], as_index=False)['SHD_pattern'].agg(['mean', 'sem']).reset_index()

,graph_p,graph_d,graph_type,data_sem,data_n,mean,sem
0,10,1,er,gp,1000,1.7,0.650641
1,10,1,er,gp,200,4.5,1.002774
2,10,1,er,gumbel,1000,8.4,0.921352
3,10,1,er,gumbel,200,9.3,1.054619
4,10,1,sf,gp,1000,1.7,0.517472
5,10,1,sf,gp,200,3.9,0.674125
6,10,1,sf,gumbel,1000,8.5,1.681600
7,10,1,sf,gumbel,200,6.1,1.494062
8,10,4,er,gp,1000,23.6,1.203698
9,10,4,er,gp,200,31.6,1.097472
